In [ ]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_set, desc

In [ ]:
spark = SparkSession.builder.appName('main').getOrCreate()

In [ ]:
dataset_path = 'inputs/by-product.csv'
output_path = 'outputs/by-product'

In [ ]:
def check(df):
    print(f"Count: {df.count()}")
    print('\nElements:')
    print(df.show(5, truncate = False))

In [ ]:
df = spark.read.options(header = True).csv(dataset_path)

In [ ]:
# check(df)

In [ ]:
df = df.groupBy('sale_code').agg(collect_set('barcode'))

In [ ]:
df = df.withColumnRenamed('collect_set(barcode)', 'items')


In [ ]:
check(df)

In [ ]:
fp_growth = FPGrowth(itemsCol = 'items', minSupport = 0.0005, minConfidence = 0.001)

In [ ]:
model = fp_growth.fit(df)

In [ ]:
freq_itemsets = model.freqItemsets

In [ ]:
freq_itemsets = freq_itemsets.select('freq', 'items')

In [ ]:
freq_itemsets = freq_itemsets.sort(col('freq').desc())

In [ ]:
# check(freq_itemsets)

In [ ]:
rules = model.associationRules

In [ ]:
rules = rules.sort(col('confidence').desc(), col('lift').desc())

In [ ]:
# check(rules)

In [ ]:
transformed = model.transform(df)

In [ ]:
# check(transformed)

In [ ]:
freq_itemsets.toPandas().to_csv(f"{output_path}/frequent_itemsets.csv", index = False)

In [ ]:
rules.toPandas().to_csv(f"{output_path}/association_rules.csv", index = False)

In [ ]:
transformed.toPandas().to_csv(f"{output_path}/transformed.csv", index = False)